In [1]:
import json
from datetime import datetime
import numpy as np
from copy import copy

channels_names = ['Fp1','Fp2','F7','F3','Fz','F4','F8','T7','T8','P7','P3','Pz','P4','P8','O1','O2']

# Trial-based Database

## Creating a Trial-based database

In this section, we will walk through the process of creating a trial-based database. First, you need to set up a test database.

First, you need to set up a test database. Import the required libraries and create an API instance:

In [2]:
from dunderlab.api import aioAPI as API
from dunderlab.api.utils import JSON

api = API('http://localhost:8000/timescaledbapp/')

Register a new source for the database:

In [3]:
source_response = await api.source.post({
    'label': 'Test.v2',
    'name': 'Test Database',
    'location': 'Eje Cafetero',
    'device': 'None',
    'protocol': 'None',
    'version': '0.1',
    'description': 'Sample trial-based database for TimeScaleDBApp',
})

JSON(source_response)


{
  "label": "Test.v2",
  "name": "Test Database",
  "location": "Eje Cafetero",
  "device": "None",
  "protocol": "None",
  "version": "0.1",
  "description": "Sample trial-based database for TimeScaleDBApp",
  "created": "2023-06-06T03:37:32.561508Z",
}


Register a new measure:

In [4]:
measure_response = await api.measure.post({
    'source': 'Test.v2',
    'label': 'measure_02',
    'name': 'Measure 02',
    'description': 'Simple sinusoidals for 64 channels at different frequencies',
})

JSON(measure_response)


{
  "label": "measure_02",
  "name": "Measure 02",
  "description": "Simple sinusoidals for 64 channels at different frequencies",
  "source": "Test.v2",
}


Register the channels:

In [5]:
channels_names = ['Fp1','Fp2','F7','F3','Fz','F4','F8','T7','T8','P7','P3','Pz','P4','P8','O1','O2']

channel_response = await api.channel.post([{
    'source': 'Test.v2',
    'measure': 'measure_02',
    'name': f'Channel {channel}',
    'label': f'{channel}',
    'unit': 'u',
    'sampling_rate': '1000',
} for channel in channels_names])

JSON(channel_response)

[
  {
    "label": "Fp1",
    "name": "Channel Fp1",
    "unit": "u",
    "sampling_rate": 1000.0,
    "description": null,
    "measure": "measure_02",
    "source": "Test.v2",
  }, 
  {
    "label": "Fp2",
    "name": "Channel Fp2",
    "unit": "u",
    "sampling_rate": 1000.0,
    "description": null,
    "measure": "measure_02",
    "source": "Test.v2",
  }, 
  {
    "label": "F7",
    "name": "Channel F7",
    "unit": "u",
    "sampling_rate": 1000.0,
    "description": null,
    "measure": "measure_02",
    "source": "Test.v2",
  }, 
  {
    "label": "F3",
    "name": "Channel F3",
    "unit": "u",
    "sampling_rate": 1000.0,
    "description": null,
    "measure": "measure_02",
    "source": "Test.v2",
  }, 
  {
    "label": "Fz",
    "name": "Channel Fz",
    "unit": "u",
    "sampling_rate": 1000.0,
    "description": null,
    "measure": "measure_02",
    "source": "Test.v2",
  }, ...]


Now that you have set up the test database and registered the required components (source, measure, and channels), you can proceed with uploading time series data and creating trials. After uploading the data, you can query the trials and reconstruct the data for further analysis. The previous code snippets you provided demonstrate how to perform these tasks.

## Creating data structure and class vector

In this section, we will create the required data structure, which is a three-dimensional vector (trial, channel, time), and a vector of classes. The three-dimensional vector represents the number of trials, channels, and time samples in the dataset. The class vector contains the class labels for each trial.

In [6]:
trials_per_class = 3
classes = 4
raw_data = np.random.normal(size=(trials_per_class*classes, len(channels_names), 1000)) # trials, channels, time

classes = np.array([f'cLass-{cls}' for cls in range(classes)] *  trials_per_class)
np.random.shuffle(classes)

raw_data.shape, classes.shape

((12, 16, 1000), (12,))

This code snippet demonstrates how to generate random data with a shape of (10, 16, 1000) representing 10 trials, 16 channels, and 1000 time samples. The trials are equally divided between two classes. The data is generated using the NumPy library and the random normal distribution function, which creates an array with the specified shape.

In addition to the data, a class vector is created with 10 elements. The vector assigns the first 5 trials to class 0 and the remaining 5 trials to class 1. The class vector will be used later to associate each trial with its corresponding class when analyzing the data.

## Uploading trials

In this section, we will demonstrate how to upload data to the database, including the new `trial` and `trial_id` arguments. These arguments are used to associate each time series data point with a specific trial and its corresponding class.

In [7]:
data = []
for i, (trial, class_) in enumerate(zip(raw_data, classes)):
    data.append({
        'source': 'Test.v2',
        'measure': 'measure_02',
        'timestamps': np.linspace(i, i+1, 1000, endpoint=False).tolist(),
        'chunk': class_,
        'values': {ch: v.tolist() for ch, v in zip(channels_names, trial)}
    })
    
JSON(data[:3])

[
  {
    "source": "Test.v2",
    "measure": "measure_02",
    "timestamps": [0.0, 0.001, 0.002, 0.003, 0.004, ...],
    "chunk": "cLass-1",
    "values": 
    {
      "Fp1": [-1.5177400358889044, -0.788977172993793, -0.03230030278992793, 0.41776901531361754, -0.7912434982958045, ...],
      "Fp2": [0.7053272431256052, 0.564824599958775, 0.24947113891302522, -0.2770154941091011, -0.31160444728795944, ...],
      "F7": [0.9545144291923049, 1.7277966884063762, -1.6484868624637383, 0.016119001124564408, -0.4861008412875246, ...],
      "F3": [-0.9227370477718901, -0.39048273125408756, -0.3404258709300914, 0.2108472949983979, -0.6825153266253882, ...],
      "Fz": [-2.2721685600289145, 1.6465217890489954, 2.4536740401499553, 0.4587805541416744, 0.5222498770782393, ...],
      "F4": [-2.065727623114571, -0.6137497754493103, -0.5120253092698225, -1.2061519414122859, -0.5172526991706605, ...],
      "F8": [-0.2354701771428703, -0.05788085719708232, 1.4967474968045162, 0.2977586287358451, -1.

This code snippet demonstrates how to create a list of time series data, including the new `trial` and `trial_id` arguments. The `trial_id` is incremented for each trial in the data, and the `trial` argument is assigned either 'Right' or 'Left' based on the class. The resulting `time_series` list contains data points with the necessary information for each trial, channel, and time sample.

In this section, we will upload the time series data to the database, following the same procedure as before.

In [8]:
await api.timeserie.post(data, batch_size=32)

[[{'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status': 'success',
   'message': 'Your data has been successfully saved.',
   'objects_created': 16000},
  {'status

This code snippet demonstrates how to upload the time series data to the database using the `api.timeserie.post()` method with a specified batch size of 32. The batch size determines how many data points are uploaded to the database in a single request, which can help improve the efficiency of the upload process.

## Querying trials

In this section, we will demonstrate how to query the trials based on certain parameters. These parameters can be used to filter and retrieve specific data from the database. In this example, we will focus on querying trials based on the 'OpenBCI' source and 'Left' and 'Right' trial classes.

In [9]:
trials_response = await api.timeserie.get({
    'source': 'Test.v2',
    'measure': 'measure_02',
    'chunks': ['cLass-0', 'cLass-1', 'cLass-2'],
    'channels': [
        'Fp1',
        'Fp2',
    ],
    'timestamps': 'false',
    # 'page_size': 2,
})

JSON(trials_response, max_list_len=3)


{
  "count": 9,
  "next": null,
  "previous": null,
  "results": [
    {
      "source": "Test.v2",
      "measure": "measure_02",
      "timestamps": [],
      "values": 
      {
        "Fp1": [-1.5177400358889044, -0.788977172993793, -0.03230030278992793, ...],
        "Fp2": [0.7053272431256052, 0.564824599958775, 0.24947113891302522, ...],
      },
      "chunk": "cLass-1",
    }, 
    {
      "source": "Test.v2",
      "measure": "measure_02",
      "timestamps": [],
      "values": 
      {
        "Fp1": [-1.3749966783068264, 0.9200507823249415, -0.6619644915989598, ...],
        "Fp2": [-0.43236545715519153, -1.243815552719948, -0.3424818244410737, ...],
      },
      "chunk": "cLass-1",
    }, 
    {
      "source": "Test.v2",
      "measure": "measure_02",
      "timestamps": [],
      "values": 
      {
        "Fp1": [-0.9630703498756138, -2.2598214681790183, -1.0793328027824352, ...],
        "Fp2": [-1.2945145713763562, 0.1693452110584934, -0.5178774846086539, ...],
  

In this example, the trials are queried based on the 'OpenBCI' source and 'Left' and 'Right' trial classes. The `channel` parameter is commented out, meaning that all channels will be included in the query. The `time` parameter is set to 'false' to exclude time information from the response. The resulting `trials_response` contains a list of trials with the specified filtering parameters applied, providing a convenient way to analyze and process the data.

## Reconstructing data from queried trials

To reconstruct the data, we will iterate through the trials in the response and extract the channel values and trial classes. This process allows us to reassemble the data into a suitable format for further analysis or processing.

In [10]:
trials = []
classes = []
for trial in trials_response['results']:
    trials.append(list(trial['values'].values()))
    classes.append(trial['chunk'])
    
np.array(trials).shape, np.array(classes).shape

((9, 2, 1000), (9,))

### Utilizing the ```get_data``` function from the Dunderlab API
The script below uses the ```get_data``` function from the Dunderlab API to achieve the same goal as the code above. This function simplifies the process of reconstructing the data from the queried trials.

In [11]:
from dunderlab.api.utils import get_data

trials, classes = get_data(trials_response)
trials.shape, classes.shape

((9, 2, 1000), (9,))

In this example, the reconstructed data has a shape of (10, 16, 1000), representing 10 trials, each with 16 channels and 1000 time points. The reconstructed classes array has a shape of (10,), indicating 10 trial classes. This reconstructed data can now be used for further analysis or processing, such as machine learning or visualization tasks.